## timediff2Rank

basedon: 11.OracleRank/laptime_rank_timediff_dataset-oracle.ipynb

rank prediction by timediff forecasting models

In [11]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import mxnet as mx
from mxnet import gluon
import pickle
import json
from scipy import stats
from sklearn.metrics import mean_squared_error
from gluonts.dataset.common import ListDataset
from gluonts.dataset.util import to_pandas
from pathlib import Path
from gluonts.model.deepar import DeepAREstimator
from gluonts.model.deep_factor import DeepFactorEstimator
from gluonts.model.deepstate import DeepStateEstimator
from gluonts.trainer import Trainer
from gluonts.model.simple_feedforward import SimpleFeedForwardEstimator
from gluonts.evaluation.backtest import make_evaluation_predictions
from gluonts.evaluation import Evaluator, MultivariateEvaluator
from gluonts.distribution.multivariate_gaussian import MultivariateGaussianOutput
from gluonts.model.predictor import Predictor
from gluonts.model.prophet import ProphetPredictor
from gluonts.model.r_forecast import RForecastPredictor
from indycar.model.NaivePredictor import NaivePredictor
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

In [2]:
import os
os.getcwd()
GPUID = 1

In [3]:
def nan_helper(y):
    """Helper to handle indices and logical indices of NaNs.

    Input:
        - y, 1d numpy array with possible NaNs
    Output:
        - nans, logical indices of NaNs
        - index, a function, with signature indices= index(logical_indices),
          to convert logical indices of NaNs to 'equivalent' indices
    Example:
        >>> # linear interpolation of NaNs
        >>> nans, x= nan_helper(y)
        >>> y[nans]= np.interp(x(nans), x(~nans), y[~nans])
    """

    return np.isnan(y), lambda z: z.nonzero()[0]

def test_flag(a, bitflag):
    return (a & bitflag) ==  bitflag

#
# remove NaN at the tail
# there should be no nans in the middle of the ts
COL_LAPTIME=0
COL_RANK=1
COL_TRACKSTATUS=2
COL_LAPSTATUS=3
COL_TIMEDIFF=4
MODE_ORACLE = 0
MODE_NOLAP = 1
MODE_NOTRACK = 2
MODE_TESTZERO = 4
MODE_TESTCURTRACK = 8
#MODE_STR={MODE_ORACLE:'oracle', MODE_NOLAP:'nolap',MODE_NOTRACK:'notrack',MODE_TEST:'test'}

def make_dataset(runs, prediction_length, freq, 
                       useeid = False,
                       run_ts=COL_LAPTIME, 
                       train_ratio = 0.8,
                       use_global_dict = True,
                       oracle_mode = MODE_ORACLE,
                       test_cars = [],
                       half_moving_win = True 
                ):
    """
    split the ts to train and test part by the ratio
    
    oracle_mode: false to simulate prediction in real by 
        set the covariates of track and lap status as nan in the testset
            
    
    """    
    start = pd.Timestamp("01-01-2019", freq=freq)  # can be different for each time series

    train_set = []
    test_set = []
    
    #select run
    if runs>=0:
        _laptime_data = [laptime_data[runs].copy()]
    else:
        _laptime_data = laptime_data.copy()
    
   
    #_data: eventid, carids, datalist[carnumbers, features, lapnumber]->[laptime, rank, track, lap]]
    for _data in _laptime_data:
        _train = []
        _test = []
        
        #statistics on the ts length
        ts_len = [ _entry.shape[1] for _entry in _data[2]]
        max_len = int(np.max(ts_len))
        train_len = int(np.max(ts_len) * train_ratio)
        
        print(f'====event:{events[_data[0]]}, train_len={train_len}, max_len={np.max(ts_len)}, min_len={np.min(ts_len)}')
                
        # process for each ts
        for rowid in range(_data[2].shape[0]):
            # rec[features, lapnumber] -> [laptime, rank, track_status, lap_status,timediff]]
            rec = _data[2][rowid].copy()
            
            #remove nan(only tails)
            nans, x= nan_helper(rec[run_ts,:])
            nan_count = np.sum(nans)             
            rec = rec[:, ~np.isnan(rec[run_ts,:])]
            
            # remove short ts
            totallen = rec.shape[1]
            if ( totallen < train_len + prediction_length):
                print(f'a short ts: carid={_data[1][rowid]}，len={totallen}')
                continue                
            
            if use_global_dict:
                carno = _data[1][rowid]
                carid = global_carids[_data[1][rowid]]
            else:
                #simulation dataset, todo, fix the carids as decoder
                carno = rowid
                carid = rowid

            #eval on carids
            if test_cars and (carno not in test_cars):
                continue                
            
            if useeid:
                static_cat = [carid, _data[0]]    
            else:
                static_cat = [carid]    
                
            # selection of features
            if test_flag(oracle_mode, MODE_NOTRACK):                
                rec[COL_TRACKSTATUS, :] = 0
            if test_flag(oracle_mode, MODE_NOLAP):                
                rec[COL_LAPSTATUS, :] = 0
                
            # split and add to dataset record
            _train.append({'target': rec[run_ts,:train_len].astype(np.float32), 
                            'start': start, 
                            'feat_static_cat': static_cat,
                            'feat_dynamic_real': [rec[COL_TRACKSTATUS,:train_len],
                                   rec[COL_LAPSTATUS,:train_len]]
                          }
                          )
            
            # multiple test ts(rolling window as half of the prediction_length)
            test_rec_cnt = 0
            step = -int(prediction_length/2) if half_moving_win else -prediction_length
            for endpos in range(max_len, train_len+prediction_length, step):
                
                
                #check if enough for this ts
                if endpos > totallen:
                    continue
                        
                track_rec = rec[COL_TRACKSTATUS, :endpos].copy()
                lap_rec = rec[COL_LAPSTATUS, :endpos].copy()
                
                # test mode
                if test_flag(oracle_mode, MODE_TESTCURTRACK):
                    # since nan does not work, use cur-val instead
                    track_rec[-prediction_length:] = track_rec[-prediction_length - 1]
                    #track_rec[-prediction_length:] = random.randint(0,1)
                    #lap_rec[-prediction_length:] = lap_rec[-prediction_length - 1]
                    lap_rec[-prediction_length:] = 0
                elif test_flag(oracle_mode, MODE_TESTZERO):
                    #set prediction part as nan
                    #track_rec[-prediction_length:] = np.nan
                    #lap_rec[-prediction_length:] = np.nan
                    track_rec[-prediction_length:] = 0
                    lap_rec[-prediction_length:] = 0                    
                
                _test.append({'target': rec[run_ts,:endpos].astype(np.float32), 
                            'start': start, 
                            'feat_static_cat': static_cat,
                            'feat_dynamic_real': [track_rec,lap_rec]
                            #'feat_dynamic_real': [rec[COL_TRACKSTATUS,:endpos],
                            #       rec[COL_LAPSTATUS,:endpos]] 
                             }
                          )   
                test_rec_cnt += 1
            
            #add one ts
            print(f'carno:{carno}, totallen:{totallen}, nancount:{nan_count}, test_reccnt:{test_rec_cnt}')

        train_set.extend(_train)
        test_set.extend(_test)

    print(f'prediction_length:{prediction_length},train len:{len(train_set)}, test len:{len(test_set)}')
    
    train_ds = ListDataset(train_set, freq=freq)
    test_ds = ListDataset(test_set, freq=freq)    
    
    return train_ds, test_ds, train_set, test_set

def save_dataset(datafile,freq, prediction_length, cardinality, train_ds, test_ds):
    with open(datafile, 'wb') as f:
        #pack [global_carids, laptime_data]
        savedata = [freq, prediction_length, cardinality, train_ds, test_ds]
        #savedata = [freq, train_set, test_set]
        # Pickle the 'data' dictionary using the highest protocol available.
        pickle.dump(savedata, f, pickle.HIGHEST_PROTOCOL)        

### test for Indy500

In [4]:
def predict(test_ds,predictor):
    forecast_it, ts_it = make_evaluation_predictions(
        dataset=test_ds,  # test dataset
        predictor=predictor,  # predictor
        num_samples=100,  # number of sample paths we want for evaluation
    )

    forecasts = list(forecast_it)
    tss = list(ts_it)
    print(f'tss len={len(tss)}, forecasts len={len(forecasts)}')
    
    return tss, forecasts

def run_prediction(test_ds, prediction_length):
    with mx.Context(mx.gpu(1)):    
        pred_ret = {}

        # deepAR-Oracle
        model_name = 'deepAR-Oracle-curtrack'
        model=f'deepAR-Oracle-timediff-curtrack-indy-f1min-t{prediction_length}-e1000-r1'
        modeldir = '../models/' + model
        print(f'predicting model={model_name}, plen={prediction_length}')
        predictor =  Predictor.deserialize(Path(modeldir))
        print(f'loading model...done!, ctx:{predictor.ctx}')
        tss, forecasts = predict(test_ds,predictor)
        pred_ret[model_name] = [tss, forecasts]

        # deepAR-Oracle
        model_name = 'deepAR-Oracle'
        model=f'deepAR-Oracle-timediff-all-indy-f1min-t{prediction_length}-e1000-r1'
        modeldir = '../models/' + model
        print(f'predicting model={model_name}, plen={prediction_length}')
        predictor =  Predictor.deserialize(Path(modeldir))
        print(f'loading model...done!, ctx:{predictor.ctx}')
        tss, forecasts = predict(test_ds,predictor)
        pred_ret[model_name] = [tss, forecasts]

        # deepAR
        model_name = 'deepAR'
        model=f'deepAR-timediff-all-indy-f1min-t{prediction_length}-e1000-r1'
        modeldir = '../models/' + model
        print(f'predicting model={model_name}, plen={prediction_length}')
        predictor =  Predictor.deserialize(Path(modeldir))
        print(f'loading model...done!, ctx:{predictor.ctx}')
        tss, forecasts = predict(test_ds,predictor)
        pred_ret[model_name] = [tss, forecasts]

        # naive
        model_name = 'naive'
        print(f'predicting model={model_name}, plen={prediction_length}')
        predictor =  NaivePredictor(freq= freq, prediction_length = prediction_length)
        tss, forecasts = predict(test_ds,predictor)
        pred_ret[model_name] = [tss, forecasts]

        # arima
        model_name = 'arima'
        print(f'predicting model={model_name}, plen={prediction_length}')
        predictor =  RForecastPredictor(method_name='arima',freq= freq, 
                                        prediction_length = prediction_length,trunc_length=60)
        #tss, forecasts = predict(test_ds,predictor)
        pred_ret[model_name] = [tss, forecasts]

        return pred_ret

In [5]:
#calc rank
def eval_rank_bytimediff(test_ds,tss,forecasts,prediction_length):
    """
    timediff models
    
    works for one event only
    
    """

    carlist = []

    # carno-lap# -> elapsed_time[] array
    forecasts_et = dict()

    ds_iter =  iter(test_ds)
    for idx in range(len(test_ds)):
        test_rec = next(ds_iter)
        #global carid
        carno = decode_carids[test_rec['feat_static_cat'][0]]
        #print('car no:', carno)

        if carno not in carlist:
            carlist.append(carno)

        # calc elapsed time
        prediction_len = forecasts[idx].samples.shape[1]
        if prediction_length != prediction_len:
            print('error: prediction_len does not match, {prediction_length}:{prediction_len}')
            return []
        
        #forecast_laptime_mean = np.mean(forecasts[idx].samples, axis=0).reshape((prediction_len,1))
        forecast_laptime_mean = np.median(forecasts[idx].samples, axis=0).reshape((prediction_len,1))
        
        timediff_array = tss[idx].values.copy()

        #save the prediction
        completed_laps = len(tss[idx]) - prediction_len + 1
        #print('car no:', carno, 'completed_laps:', completed_laps)
        #key = '%s-%s'%(carno, completed_laps)
        #forecasts_et[key] = elapsed_time[-prediction_len:].copy()
        if completed_laps not in forecasts_et:
            forecasts_et[completed_laps] = {}
        forecasts_et[completed_laps][carno] = [timediff_array[-prediction_len:].copy(),
                                                   forecast_laptime_mean.copy()]


    # calc rank
    rank_ret = []
    for lap in forecasts_et.keys():
        #get car list for this lap
        carlist = list(forecasts_et[lap].keys())
        #print('carlist:', carlist)

        caridmap={key:idx for idx, key in enumerate(carlist)}

        #fill in data
        time_diff = np.zeros((2, len(carlist), prediction_len))
        for carno in carlist:
            carid = caridmap[carno]
            time_diff[0, carid, :] = forecasts_et[lap][carno][0].reshape((prediction_len))
            time_diff[1, carid, :] = forecasts_et[lap][carno][1].reshape((prediction_len))

        #calculate rank    
        idx = np.argsort(time_diff[0], axis=0)
        true_rank = np.argsort(idx, axis=0)

        idx = np.argsort(time_diff[1], axis=0)
        pred_rank = np.argsort(idx, axis=0)

        rank_ret.append([lap, time_diff, true_rank, pred_rank])
        
    return rank_ret,forecasts_et
    
   
   
def get_acc(rank_ret,prediction_length):    
    # evaluate
    #top1 accuracy
    top1acc = 0
    top1acc_farmost = 0
    top5acc = 0
    top5acc_farmost = 0
    tau = 0
    rmse = 0.
    
    for rec in rank_ret:
        trueRank = rec[2]
        predRank = rec[3]

        #top1 , rank = 0, first col is not prediction
        top1acc += np.sum((trueRank==0) & (predRank==0)) 
        
        top1acc_farmost += np.sum((trueRank[:,-1]==0) & (predRank[:,-1]==0))
        
        #top5
        top5acc += np.sum((trueRank<5) & (predRank<5)) 
        
        top5acc_farmost += np.sum((trueRank[:,-1]<5) & (predRank[:,-1]<5))
        
        # tau
        tao, _ = stats.kendalltau(trueRank, predRank)
        tau += tao
        
        #rmse
        rmse += mean_squared_error(predRank,trueRank)
        

    print(f'total:{len(rank_ret)}, prediction_length:{prediction_length}') 
    print('top1acc=', top1acc *1.0/ (len(rank_ret)*prediction_length),
          'top1acc_farmost=', top1acc_farmost *1.0/ (len(rank_ret)),
          'top5acc=', top5acc *1.0/ (5*len(rank_ret)*prediction_length),
          'top5acc_farmost=', top5acc_farmost *1.0/ (5*len(rank_ret)),
         )
    print('tau = ', tau/len(rank_ret),
         'rmse = ', rmse/len(rank_ret))
    
    
def get_top1acc_farmost(rank_ret,prediction_len):    
    # evaluate
    #top1 accuracy
    hitcnt = 0
    for rec in rank_ret:
        trueRank = rec[2]
        predRank = rec[3]

        #top1 , rank = 0, first col is not prediction
        hitcnt += np.sum((trueRank[:,-1]==0) & (predRank[:,-1]==0)) 

    print('total:', hitcnt, 'top1acc_farmost=', hitcnt *1.0/ (len(rank_ret)*prediction_length))    



In [6]:
def run_exp(prediction_length, half_moving_win):
    ### create test dataset
    test_cars = []
    train_ds, test_ds,_,_ = make_dataset(-1, prediction_length,freq, 
                                         oracle_mode=MODE_TESTCURTRACK,
                                         run_ts = COL_TIMEDIFF,
                                         test_cars=test_cars,
                                         half_moving_win= half_moving_win)
    pred_ret = run_prediction(test_ds, prediction_length)

    models = ['deepAR-Oracle','deepAR-Oracle-curtrack','deepAR','naive','arima']
    for model in models:
        print('model:', model)
        tss, forecasts = pred_ret[model]

        rank_ret,_ = eval_rank_bytimediff(test_ds,tss,forecasts,prediction_length)
        get_acc(rank_ret,prediction_length)
        
    return pred_ret

### init

In [7]:
#
# parameters
#
#year = '2017'
year = '2018'
#event = 'Toronto'
#https://www.racing-reference.info/season-stats/2018/O/#
events_totalmiles=[256,500,372,268,500,310]
events_laplen = [1.022,2.5,1.5,0.894,2.5,1.25]
events = ['Phoenix','Indy500','Texas','Iowa','Pocono','Gateway']
#events = ['Gateway']

#events = ['Indy500']
#events = ['Phoenix']
events_id={key:idx for idx, key in enumerate(events)}

In [8]:
# start from here
import pickle
with open('laptime_rank_timediff-oracle-%s.pickle'%year, 'rb') as f:
    # The protocol version used is detected automatically, so we do not
    # have to specify it.
    global_carids, laptime_data = pickle.load(f, encoding='latin1')

In [9]:
freq = "1min"
#decode global_carids
decode_carids={carid:carno for carno, carid in global_carids.items()}
    
#useeid = False
#interpolate = False
#ipstr = '-ip' if interpolate else '-noip'
#ipstr = '%s-%s'%('ip' if interpolate else 'noip', 'eid' if useeid else 'noeid')
#if useeid:
#    cardinality = [len(global_carids), len(laptime_data)]
#else:
#    cardinality = [len(global_carids)]

#### test 2

In [12]:
# test half moving win
pred_ret2_nohalfwin = run_exp(2, False)

====event:Phoenix, train_len=200, max_len=250, min_len=250
carno:1, totallen:250, nancount:0, test_reccnt:24
carno:4, totallen:241, nancount:9, test_reccnt:19
carno:5, totallen:250, nancount:0, test_reccnt:24
carno:6, totallen:250, nancount:0, test_reccnt:24
carno:9, totallen:250, nancount:0, test_reccnt:24
carno:10, totallen:229, nancount:21, test_reccnt:13
a short ts: carid=12，len=154
carno:14, totallen:250, nancount:0, test_reccnt:24
carno:15, totallen:250, nancount:0, test_reccnt:24
carno:18, totallen:249, nancount:1, test_reccnt:23
a short ts: carid=19，len=40
carno:20, totallen:250, nancount:0, test_reccnt:24
carno:21, totallen:249, nancount:1, test_reccnt:23
carno:22, totallen:250, nancount:0, test_reccnt:24
carno:23, totallen:248, nancount:2, test_reccnt:23
carno:26, totallen:249, nancount:1, test_reccnt:23
carno:27, totallen:250, nancount:0, test_reccnt:24
carno:28, totallen:250, nancount:0, test_reccnt:24
carno:30, totallen:250, nancount:0, test_reccnt:24
a short ts: carid=32，

INFO:root:Using GPU


predicting model=deepAR-Oracle-curtrack, plen=2
loading model...done!, ctx:gpu(0)


INFO:root:Using GPU


tss len=2503, forecasts len=2503
predicting model=deepAR-Oracle, plen=2
loading model...done!, ctx:gpu(0)


INFO:root:Using GPU


tss len=2503, forecasts len=2503
predicting model=deepAR, plen=2
loading model...done!, ctx:gpu(0)
tss len=2503, forecasts len=2503
predicting model=naive, plen=2


/N/u/pengb/hpda/indycar/predictor/src/indycar/model/NaivePredictor.py:60: FutureWarning: Addition/subtraction of integers and integer-arrays to Timestamp is deprecated, will be removed in a future version.  Instead of adding/subtracting `n`, use `n * self.freq`
  start_date=start + target_len,


tss len=2503, forecasts len=2503
predicting model=arima, plen=2
model: deepAR-Oracle
total:69, prediction_length:2
top1acc= 0.7318840579710145 top1acc_farmost= 0.35507246376811596 top5acc= 0.936231884057971 top5acc_farmost= 0.45507246376811594
tau =  0.9496749124048769
model: deepAR-Oracle-curtrack
total:69, prediction_length:2
top1acc= 0.782608695652174 top1acc_farmost= 0.38405797101449274 top5acc= 0.9405797101449276 top5acc_farmost= 0.4579710144927536
tau =  0.9512993494033231
model: deepAR
total:69, prediction_length:2
top1acc= 0.7536231884057971 top1acc_farmost= 0.37681159420289856 top5acc= 0.9420289855072463 top5acc_farmost= 0.45942028985507244
tau =  0.954172185952718
model: naive
total:69, prediction_length:2
top1acc= 0.9057971014492754 top1acc_farmost= 0.43478260869565216 top5acc= 0.9594202898550724 top5acc_farmost= 0.46956521739130436
tau =  0.9670808917042292
model: arima
total:69, prediction_length:2
top1acc= 0.9057971014492754 top1acc_farmost= 0.43478260869565216 top5acc= 0

In [13]:
pred_ret2_halfwin = run_exp(2, True)

INFO:root:Using GPU


====event:Phoenix, train_len=200, max_len=250, min_len=250
carno:1, totallen:250, nancount:0, test_reccnt:48
carno:4, totallen:241, nancount:9, test_reccnt:39
carno:5, totallen:250, nancount:0, test_reccnt:48
carno:6, totallen:250, nancount:0, test_reccnt:48
carno:9, totallen:250, nancount:0, test_reccnt:48
carno:10, totallen:229, nancount:21, test_reccnt:27
a short ts: carid=12，len=154
carno:14, totallen:250, nancount:0, test_reccnt:48
carno:15, totallen:250, nancount:0, test_reccnt:48
carno:18, totallen:249, nancount:1, test_reccnt:47
a short ts: carid=19，len=40
carno:20, totallen:250, nancount:0, test_reccnt:48
carno:21, totallen:249, nancount:1, test_reccnt:47
carno:22, totallen:250, nancount:0, test_reccnt:48
carno:23, totallen:248, nancount:2, test_reccnt:46
carno:26, totallen:249, nancount:1, test_reccnt:47
carno:27, totallen:250, nancount:0, test_reccnt:48
carno:28, totallen:250, nancount:0, test_reccnt:48
carno:30, totallen:250, nancount:0, test_reccnt:48
a short ts: carid=32，

INFO:root:Using GPU


tss len=5045, forecasts len=5045
predicting model=deepAR-Oracle, plen=2
loading model...done!, ctx:gpu(0)


INFO:root:Using GPU


tss len=5045, forecasts len=5045
predicting model=deepAR, plen=2
loading model...done!, ctx:gpu(0)
tss len=5045, forecasts len=5045
predicting model=naive, plen=2
tss len=5045, forecasts len=5045
predicting model=arima, plen=2
model: deepAR-Oracle
total:138, prediction_length:2
top1acc= 0.75 top1acc_farmost= 0.36594202898550726 top5acc= 0.9326086956521739 top5acc_farmost= 0.45652173913043476
tau =  0.9522462234551854
model: deepAR-Oracle-curtrack
total:138, prediction_length:2
top1acc= 0.7644927536231884 top1acc_farmost= 0.37681159420289856 top5acc= 0.9384057971014492 top5acc_farmost= 0.45942028985507244
tau =  0.9535839343291898
model: deepAR
total:138, prediction_length:2
top1acc= 0.7463768115942029 top1acc_farmost= 0.36594202898550726 top5acc= 0.9333333333333333 top5acc_farmost= 0.4579710144927536
tau =  0.9538291926432702
model: naive
total:138, prediction_length:2
top1acc= 0.8876811594202898 top1acc_farmost= 0.427536231884058 top5acc= 0.9485507246376812 top5acc_farmost= 0.46739130

In [14]:
# test half moving win
pred_ret5_halfwin = run_exp(5, True)

====event:Phoenix, train_len=200, max_len=250, min_len=250
carno:1, totallen:250, nancount:0, test_reccnt:23
carno:4, totallen:241, nancount:9, test_reccnt:18
carno:5, totallen:250, nancount:0, test_reccnt:23
carno:6, totallen:250, nancount:0, test_reccnt:23
carno:9, totallen:250, nancount:0, test_reccnt:23
carno:10, totallen:229, nancount:21, test_reccnt:12
a short ts: carid=12，len=154
carno:14, totallen:250, nancount:0, test_reccnt:23
carno:15, totallen:250, nancount:0, test_reccnt:23
carno:18, totallen:249, nancount:1, test_reccnt:22
a short ts: carid=19，len=40
carno:20, totallen:250, nancount:0, test_reccnt:23
carno:21, totallen:249, nancount:1, test_reccnt:22
carno:22, totallen:250, nancount:0, test_reccnt:23
carno:23, totallen:248, nancount:2, test_reccnt:22
carno:26, totallen:249, nancount:1, test_reccnt:22
carno:27, totallen:250, nancount:0, test_reccnt:23
carno:28, totallen:250, nancount:0, test_reccnt:23
carno:30, totallen:250, nancount:0, test_reccnt:23
a short ts: carid=32，

INFO:root:Using GPU


predicting model=deepAR-Oracle-curtrack, plen=5
loading model...done!, ctx:gpu(0)


INFO:root:Using GPU


tss len=2387, forecasts len=2387
predicting model=deepAR-Oracle, plen=5
loading model...done!, ctx:gpu(0)


INFO:root:Using GPU


tss len=2387, forecasts len=2387
predicting model=deepAR, plen=5
loading model...done!, ctx:gpu(0)
tss len=2387, forecasts len=2387
predicting model=naive, plen=5
tss len=2387, forecasts len=2387
predicting model=arima, plen=5
model: deepAR-Oracle
total:67, prediction_length:5
top1acc= 0.7044776119402985 top1acc_farmost= 0.14029850746268657 top5acc= 0.9002985074626866 top5acc_farmost= 0.17134328358208956
tau =  0.9153215401113388
model: deepAR-Oracle-curtrack
total:67, prediction_length:5
top1acc= 0.7253731343283583 top1acc_farmost= 0.14029850746268657 top5acc= 0.8985074626865671 top5acc_farmost= 0.17253731343283582
tau =  0.9182139742423303
model: deepAR
total:67, prediction_length:5
top1acc= 0.7074626865671642 top1acc_farmost= 0.1373134328358209 top5acc= 0.8937313432835821 top5acc_farmost= 0.16955223880597015
tau =  0.9163297852774634
model: naive
total:67, prediction_length:5
top1acc= 0.8119402985074626 top1acc_farmost= 0.15223880597014924 top5acc= 0.9032835820895523 top5acc_farmost

In [15]:
# test half moving win
pred_ret5_nohalfwin = run_exp(5, False)

INFO:root:Using GPU


====event:Phoenix, train_len=200, max_len=250, min_len=250
carno:1, totallen:250, nancount:0, test_reccnt:9
carno:4, totallen:241, nancount:9, test_reccnt:7
carno:5, totallen:250, nancount:0, test_reccnt:9
carno:6, totallen:250, nancount:0, test_reccnt:9
carno:9, totallen:250, nancount:0, test_reccnt:9
carno:10, totallen:229, nancount:21, test_reccnt:4
a short ts: carid=12，len=154
carno:14, totallen:250, nancount:0, test_reccnt:9
carno:15, totallen:250, nancount:0, test_reccnt:9
carno:18, totallen:249, nancount:1, test_reccnt:8
a short ts: carid=19，len=40
carno:20, totallen:250, nancount:0, test_reccnt:9
carno:21, totallen:249, nancount:1, test_reccnt:8
carno:22, totallen:250, nancount:0, test_reccnt:9
carno:23, totallen:248, nancount:2, test_reccnt:8
carno:26, totallen:249, nancount:1, test_reccnt:8
carno:27, totallen:250, nancount:0, test_reccnt:9
carno:28, totallen:250, nancount:0, test_reccnt:9
carno:30, totallen:250, nancount:0, test_reccnt:9
a short ts: carid=32，len=174
carno:59,

INFO:root:Using GPU


tss len=905, forecasts len=905
predicting model=deepAR-Oracle, plen=5
loading model...done!, ctx:gpu(0)


INFO:root:Using GPU


tss len=905, forecasts len=905
predicting model=deepAR, plen=5
loading model...done!, ctx:gpu(0)
tss len=905, forecasts len=905
predicting model=naive, plen=5
tss len=905, forecasts len=905
predicting model=arima, plen=5
model: deepAR-Oracle
total:35, prediction_length:5
top1acc= 0.7257142857142858 top1acc_farmost= 0.13142857142857142 top5acc= 0.8742857142857143 top5acc_farmost= 0.1657142857142857
tau =  0.8980669514625466
model: deepAR-Oracle-curtrack
total:35, prediction_length:5
top1acc= 0.7314285714285714 top1acc_farmost= 0.13714285714285715 top5acc= 0.888 top5acc_farmost= 0.16685714285714287
tau =  0.895082138932046
model: deepAR
total:35, prediction_length:5
top1acc= 0.7542857142857143 top1acc_farmost= 0.14285714285714285 top5acc= 0.9097142857142857 top5acc_farmost= 0.17485714285714285
tau =  0.904176527161472
model: naive
total:35, prediction_length:5
top1acc= 0.7657142857142857 top1acc_farmost= 0.13714285714285715 top5acc= 0.9017142857142857 top5acc_farmost= 0.17257142857142857

In [16]:
pred_ret10_halfwin = run_exp(10, True)

INFO:root:Using GPU


====event:Phoenix, train_len=200, max_len=250, min_len=250
carno:1, totallen:250, nancount:0, test_reccnt:8
carno:4, totallen:241, nancount:9, test_reccnt:6
carno:5, totallen:250, nancount:0, test_reccnt:8
carno:6, totallen:250, nancount:0, test_reccnt:8
carno:9, totallen:250, nancount:0, test_reccnt:8
carno:10, totallen:229, nancount:21, test_reccnt:3
a short ts: carid=12，len=154
carno:14, totallen:250, nancount:0, test_reccnt:8
carno:15, totallen:250, nancount:0, test_reccnt:8
carno:18, totallen:249, nancount:1, test_reccnt:7
a short ts: carid=19，len=40
carno:20, totallen:250, nancount:0, test_reccnt:8
carno:21, totallen:249, nancount:1, test_reccnt:7
carno:22, totallen:250, nancount:0, test_reccnt:8
carno:23, totallen:248, nancount:2, test_reccnt:7
carno:26, totallen:249, nancount:1, test_reccnt:7
carno:27, totallen:250, nancount:0, test_reccnt:8
carno:28, totallen:250, nancount:0, test_reccnt:8
carno:30, totallen:250, nancount:0, test_reccnt:8
a short ts: carid=32，len=174
carno:59,

INFO:root:Using GPU


tss len=791, forecasts len=791
predicting model=deepAR-Oracle, plen=10
loading model...done!, ctx:gpu(0)


INFO:root:Using GPU


tss len=791, forecasts len=791
predicting model=deepAR, plen=10
loading model...done!, ctx:gpu(0)
tss len=791, forecasts len=791
predicting model=naive, plen=10
tss len=791, forecasts len=791
predicting model=arima, plen=10
model: deepAR-Oracle
total:32, prediction_length:10
top1acc= 0.5125 top1acc_farmost= 0.03125 top5acc= 0.8275 top5acc_farmost= 0.07375
tau =  0.828850462662982
model: deepAR-Oracle-curtrack
total:32, prediction_length:10
top1acc= 0.55625 top1acc_farmost= 0.03125 top5acc= 0.83625 top5acc_farmost= 0.076875
tau =  0.8507176443561308
model: deepAR
total:32, prediction_length:10
top1acc= 0.53125 top1acc_farmost= 0.0375 top5acc= 0.84625 top5acc_farmost= 0.07875
tau =  0.8391164742342568
model: naive
total:32, prediction_length:10
top1acc= 0.68125 top1acc_farmost= 0.053125 top5acc= 0.86625 top5acc_farmost= 0.080625
tau =  0.8804793956971103
model: arima
total:32, prediction_length:10
top1acc= 0.68125 top1acc_farmost= 0.053125 top5acc= 0.86625 top5acc_farmost= 0.080625
tau =

In [17]:
pred_ret10_nohalfwin = run_exp(10, False)

INFO:root:Using GPU


====event:Phoenix, train_len=200, max_len=250, min_len=250
carno:1, totallen:250, nancount:0, test_reccnt:4
carno:4, totallen:241, nancount:9, test_reccnt:3
carno:5, totallen:250, nancount:0, test_reccnt:4
carno:6, totallen:250, nancount:0, test_reccnt:4
carno:9, totallen:250, nancount:0, test_reccnt:4
carno:10, totallen:229, nancount:21, test_reccnt:1
a short ts: carid=12，len=154
carno:14, totallen:250, nancount:0, test_reccnt:4
carno:15, totallen:250, nancount:0, test_reccnt:4
carno:18, totallen:249, nancount:1, test_reccnt:3
a short ts: carid=19，len=40
carno:20, totallen:250, nancount:0, test_reccnt:4
carno:21, totallen:249, nancount:1, test_reccnt:3
carno:22, totallen:250, nancount:0, test_reccnt:4
carno:23, totallen:248, nancount:2, test_reccnt:3
carno:26, totallen:249, nancount:1, test_reccnt:3
carno:27, totallen:250, nancount:0, test_reccnt:4
carno:28, totallen:250, nancount:0, test_reccnt:4
carno:30, totallen:250, nancount:0, test_reccnt:4
a short ts: carid=32，len=174
carno:59,

INFO:root:Using GPU


tss len=371, forecasts len=371
predicting model=deepAR-Oracle, plen=10
loading model...done!, ctx:gpu(0)


INFO:root:Using GPU


tss len=371, forecasts len=371
predicting model=deepAR, plen=10
loading model...done!, ctx:gpu(0)
tss len=371, forecasts len=371
predicting model=naive, plen=10
tss len=371, forecasts len=371
predicting model=arima, plen=10
model: deepAR-Oracle
total:16, prediction_length:10
top1acc= 0.475 top1acc_farmost= 0.03125 top5acc= 0.8075 top5acc_farmost= 0.07125
tau =  0.7859952564678183
model: deepAR-Oracle-curtrack
total:16, prediction_length:10
top1acc= 0.5375 top1acc_farmost= 0.0375 top5acc= 0.81875 top5acc_farmost= 0.075
tau =  0.80675895458171
model: deepAR
total:16, prediction_length:10
top1acc= 0.54375 top1acc_farmost= 0.0375 top5acc= 0.82125 top5acc_farmost= 0.0775
tau =  0.8095001258387868
model: naive
total:16, prediction_length:10
top1acc= 0.675 top1acc_farmost= 0.05625 top5acc= 0.8475 top5acc_farmost= 0.08
tau =  0.843276627076627
model: arima
total:16, prediction_length:10
top1acc= 0.675 top1acc_farmost= 0.05625 top5acc= 0.8475 top5acc_farmost= 0.08
tau =  0.843276627076627


### test

In [ ]:
pred_ret = pred_ret2_halfwin
model = 'deepAR-Oracle'
tss, forecasts = pred_ret[model]
rank_ret = eval_rank_bytimediff()

In [ ]:
tss, forecasts = pred_ret5_halfwin['deepAR-Oracle']
rank_ret = eval_rank_bytimediff(test_ds,tss,forecasts,5)
get_acc(rank_ret,5)

In [ ]:
%debug

In [ ]:
len(test_ds)

In [ ]:
rank_ret[1][2][:,0:4]

In [ ]:
rank_ret[1][3][:,:]

In [ ]:
tss, forecasts = pred_ret['deepAR-Oracle-curtrack']
forecasts[0].samples.shape[1]

In [ ]:
prediction_length

In [ ]:
pred_ret.keys()

In [ ]:
tss, forecasts = 0, 0
model = 'deepAR-Oracle-curtrack'

tss, forecasts = pred_ret2[model]

rank_ret = eval_rank_bytimediff()
get_acc()

In [ ]:
rank_ret[0]

In [ ]:
forecasts[0].samples.shape

In [ ]:
next(iter(test_ds))